# Image Acquiring Utilizing Google Maps Static API
In order to use it you need to enable Google Maps Static API from https://console.cloud.google.com/google/maps-apis and create an API key.

Google Maps API Request maker, used for acquiring images of utility poles.
CSV files contain geographic locations of different types of poles, only
utility poles are used. The acquired images were manually uploaded to the roboflow dataset for annotation.

https://app.roboflow.com/telephonepoledetection/telephone-poles/deploy

In [25]:
import requests
import pandas as pd
from PIL import Image
import os
import numpy as np
from io import BytesIO
import cv2 as cv
from matplotlib import pyplot as plt

In [19]:

os.makedirs('./dataset/california/',exist_ok=True)

os.makedirs('./dataset/dc/', exist_ok=True)

os.makedirs('./dataset/uk/', exist_ok=True)

os.makedirs('./dataset/test/', exist_ok=True)
dataset_path_cali = './dataset/california/'
dataset_path_dc = './dataset/dc/'

uk_dataset = pd.read_csv('ukpn-lv-poles.csv')
dataset_cali = pd.read_csv('California.csv')
dataset_dc = pd.read_csv('DC.csv')

In [7]:
dataset_cali['Type'].unique()

array(['Power Pole', 'Telecom', nan], dtype=object)

In [ ]:
telecom_dataset = dataset_cali[(dataset_cali['Type']=='Telecom')]

In [8]:
dataset_dc_utility = dataset_dc[(dataset_dc['DXF_LAYER'] == 'UTLPOLE')]

In [10]:
coordinates_cali = dataset_cali[['Y','X','Type']]
coordinates_dc = dataset_dc[['Y','X','DXF_LAYER']].rename({'DXF_LAYER':'Type'})

In [ ]:
coordinates_cali

In [13]:
def crop_bottom(image):
    w, h = image.size
    return image.crop((0, 0, w, h-20 ))


In [14]:
def make_request(x,y,path, type):
    center_string = str(x) + ',' + str(y)
    payload = {'center':center_string,'zoom':19,'size':'600x600','key':'','maptype':'satellite'}
    r = requests.get('https://maps.googleapis.com/maps/api/staticmap', params=payload)
    i = Image.open(BytesIO(r.content))
    i = crop_bottom(i)
    datapath = path + '/' + type + '_' + str(x) + '_' + str(y) + '.png'
    i.save(datapath)
    if r.status_code == 200:
        return True
    else:
        return False

In [ ]:
# Test an image
make_request(50.91811091177017, -0.24032128131631858, './', 'test')

In [26]:
uk_dataset[['X', 'Y']] = uk_dataset['Geo Point'].str.split(',', n=1, expand=True)

uk_dataset['dist'] = uk_dataset[['X', 'Y']].apply(lambda row: np.linalg.norm((row.X, row.Y)), axis=1)
uk_dataset.sort_values('dist', ignore_index=True, inplace=True)

In [ ]:
reduced_uk_dataset = uk_dataset[uk_dataset.index % 20 == 0]
for row  in reduced_uk_dataset.iloc[6000:8000,:].values:
    if not make_request(row[-1], row[-2], './dataset/uk',row[3]+'_'+row[5]):
        break

In [ ]:
reduced_uk_dataset.iloc[6000:8000,:]

In [ ]:
for row in coordinates_cali.values:
    # it's reversed for some reason
    x, y, type = row[0], row[1], row[2]
    if not make_request(x,y,dataset_path_cali,type):
        break

In [ ]:
for row in coordinates_dc.values:
    x, y, type = row[0], row[1], row[2]
    if not make_request(x,y,dataset_path_dc, type):
        break

In [ ]:
img = cv.imread('./test_50.91811091177017_-0.24032128131631858.png')
plt.plot(plt.imshow(img))

In [ ]:
img = cv.imread('./test_50.91811091177017_-0.24032128131631858.png')
assert img is not None, "file could not be read, check with os.path.exists()"

laplacian = cv.Laplacian(img,cv.CV_64F)
sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=3)
sobely = cv.Sobel(img,cv.CV_64F,0,1,ksize=3)


plt.subplot(2,2,1),plt.imshow(img)
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(2,2,2),plt.imshow(laplacian)
plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
plt.subplot(2,2,3),plt.imshow(sobelx)
plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
plt.subplot(2,2,4),plt.imshow(sobely)
plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])